Exercise 1

In [ ]:
import pandas as pd

def perform_data_checks(data):
    """
    Voert checks uit op de gegeven data.

    Parameters:
    - data (list): De input data als een list van lists.

    Returns:
    - pd.DataFrame: Het opgeschoonde DataFrame.

    """

    # Laad data in DataFrame
    df = pd.DataFrame(data)
    
    # Verwijderd alle spaties voor/na de inhoud per kolom
    df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    print("Stap 1: Spaties verwijderd")
    
    # Controleer op missende waardes
    print("Stap 2: Controleer op missende waardes")
    if df.isnull().values.any():
        print("Waarschuwing: De data bevat missende waardes.")
    else:
        print("Geen missende waardes gevonden")
        
    # Controleren op dubbele rijen
    print("Stap 3: Controleren op dubbele rijen")
    if df.duplicated().any():
        print("Waarschuwing: De data bevat dubbele rijen.")
    else:
        print("Geen dubbele rijen gevonden")
        
    # Maakt kolommen van type string hoofdletter
    string_cols = df.select_dtypes(include=['object']).columns
    df[string_cols] = df[string_cols].apply(lambda x: x.str.upper() if x.dtype == "object" else x)
    print("Stap 4: Strings vervangen door hoofdletters")
    
    # Controleert op inconsistente data types per rij
    print("Stap 5: Controleert op inconsistente data types per rij.")
    inconsistent_columns = []
    for col in df.columns:
        unique_types = df[col].apply(type).unique()
        if len(unique_types) > 1:
            inconsistent_columns.append(col)
    if inconsistent_columns:
        print(f"Waarschuwing: De data bevat inconsistente data types in de volgende kolommen: {', '.join(inconsistent_columns)}.")
    else:
        print("Geen inconsistente datatypes gevonden.")
    
    # Return het opgeschoonde DataFrame
    return df


# Voorbeeld
file_path = 'dataProject4.xlsx'
data = pd.read_excel(file_path, sheet_name='20000-211000')
checks= perform_data_checks(data)

Exercise 3

In [ ]:
import pandas as pd

def generate_sales_report(file_path):
    """
    Generates a sales report based on data from an Excel file.

    Args:
        file_path (str): The path to the Excel file.

    Returns:
        pd.DataFrame: The sales report containing total sales and contribution for categories, months, and managers.
    """
    # Excel bestand lezen
    data = pd.read_excel(file_path)

    # Calculate total sales and contribution by category
    category_sales = data.groupby('Category')['Sales'].sum().dropna()
    total_sales_category = category_sales.sum()
    category_contributions = category_sales / total_sales_category * 100

    # Calculate total sales and contribution by month
    month_sales = data.groupby('Month')['Sales'].sum()
    total_sales_month = month_sales.sum()
    month_contributions = month_sales / total_sales_month * 100

    # Calculate total sales and contribution by manager
    manager_sales = data.groupby('Sales Manager')['Sales'].sum()
    total_sales_manager = manager_sales.sum()
    manager_contributions = manager_sales / total_sales_manager * 100

    # Generate the sales report
    report = pd.DataFrame({
        'Total Sales by Category': category_sales,
        'Contribution Category (%)': category_contributions,
        'Total Sales by Month': month_sales,
        'Contribution Month (%)': month_contributions,
        'Total Sales by Manager': manager_sales,
        'Contribution Manager (%)': manager_contributions
    })

    # Sort the report by category, month, and manager
    report = report.sort_values(by=['Total Sales by Category', 'Total Sales by Month', 'Total Sales by Manager'],
                                ascending=False)

    return report
# Specify the file path of the Excel file
file_path = 'detailedRetail.xlsx'

# Generate the sales report
sales_report = generate_sales_report(file_path)

# Save the sales report to an Excel file
output_file_path = 'reportRetail.xlsx'
sales_report.to_excel(output_file_path, index=True)

# Confirm the file is saved
print(f"The sales report has been saved to '{output_file_path}'.")

Exercise 4

In [ ]:
#pip install langdetect

In [ ]:
from langdetect import detect
import pandas as pd
# Read file
tweets= pd.read_excel('tweets.xlsx')
print(tweets)
# Create a new column 'language' to store the identified language
tweets['language'] = ''

# Iterate over each tweet and detect the language
for index, row in tweets.iterrows():
    text = row['Tweet']
    try:
        language = detect(text)
        tweets.at[index, 'language'] = language
    except:
        tweets.at[index, 'language'] = 'Unknown'

# Print the updated DataFrame with the identified languages
print(tweets)

In [ ]:
pip install textblob
pip install nltk
import nltk
nltk.download('vader_lexicon')

In [ ]:
from langdetect import detect
from textblob import TextBlob
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import pandas as pd

tweets = pd.read_excel('tweets.xlsx')

# Create a new column 'language' to store the identified language
tweets['language'] = ''

# Iterate over each tweet and detect the language
for index, row in tweets.iterrows():
    text = row['Tweet']
    try:
        language = detect(text)
        tweets.at[index, 'language'] = language
    except:
        tweets.at[index, 'language'] = 'Unknown'

# Perform sentiment analysis for English tweets using TextBlob
def analyze_sentiment_english(tweet):
    blob = TextBlob(tweet)
    polarity = blob.sentiment.polarity
    if polarity > 0:
        return 'positive'
    elif polarity < 0:
        return 'negative'
    else:
        return 'neutral'

# Perform sentiment analysis for non-English tweets using NLTK's SentimentIntensityAnalyzer
def analyze_sentiment_other(tweet):
    sia = SentimentIntensityAnalyzer()
    scores = sia.polarity_scores(str(tweet))  # Convert tweet to string
    compound_score = scores['compound']
    if compound_score >= 0.05:
        return 'positive'
    elif compound_score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

# Apply language-specific sentiment analysis to each tweet
tweets['sentiment'] = ''
for index, row in tweets.iterrows():
    tweet = row['Tweet']
    language = row['language']
    if language == 'en':
        sentiment = analyze_sentiment_english(tweet)
    else:
        sentiment = analyze_sentiment_other(str(tweet))  # Convert tweet to string
    tweets.at[index, 'sentiment'] = sentiment

# Print the updated DataFrame with the sentiment analysis results
print(tweets)
